In [80]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import functools
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
import qutip as qt
import time
import operator


In [72]:
# # This dictionary maps string keys ('x', 'y', 'z', 'p', 'm', 'i') to functions that generate spin operators for a given dimension dim.
# opstr2fun = {'x': lambda dim: qt.spin_Jx((dim-1)/2),
#              'y': lambda dim: qt.spin_Jy((dim-1)/2),
#              'z': lambda dim: qt.spin_Jz((dim-1)/2),
#              'p': lambda dim: qt.spin_Jp((dim-1)/2),
#              'm': lambda dim: qt.spin_Jm((dim-1)/2),
#              'i': qt.identity}
# # Initializes ops as a list of identity matrices for each dimension in dims. Iterates over specs to replace the identity matrix at the specified index with the corresponding spin operator. Returns the tensor product of the operators in ops using qt.tensor.
# def mkSpinOp(dims, specs):
#     ops = [qt.identity(d) for d in dims]
#     for ind, opstr in specs:
#         ops[ind] = ops[ind] * opstr2fun[opstr](dims[ind])
#     return qt.tensor(ops)
# # Constructs a Hamiltonian for a single spin system with interactions along the x, y, and z axes.
# def mkH1(dims, ind, parvec):
#     axes = ['x', 'y', 'z']
#     # Creates a list of spin operators weighted by the corresponding parameters in parvec (ignores zero parameters). Uses functools.reduce to sum these weighted spin operators.
#     return functools.reduce(lambda a, b: a + b, 
#                [v * mkSpinOp(dims, [(ind,ax)]) for v, ax in zip(parvec, axes) if v!=0])
# # Constructs a Hamiltonian for the interaction between two spin systems with interaction terms along all combinations of x, y, and z axes.
# def mkH12(dims, ind1, ind2, parmat):
#     axes = ['x', 'y', 'z']
#     ops = []
#     # Iterates over all combinations of the x, y, and z axes for the two spins. For each non-zero element in parmat, adds the corresponding spin-spin interaction term to the empty list ops.
#     for i in range(3):
#         for j in range(3):
#             if parmat[i,j] != 0:
#                 ops.append(parmat[i,j] * mkSpinOp(dims, [(ind1,axes[i]), (ind2,axes[j])]))
#     return functools.reduce(lambda a, b: a + b, ops) # Uses functools.reduce to sum these interaction terms.

In [75]:
# N5_C =  2*np.pi* np.array([[-0.36082693, -0.0702137 , -1.41518116],
#       [-0.0702137 , -0.60153649,  0.32312139],
#       [-1.41518116,  0.32312139, 50.80213093]]) # in MHz
	  
# N1_C = 2*np.pi*np.array([[  2.13814981,   3.19255832,  -2.48895215],
#       [  3.19255832,  15.45032887, -12.44778343],
#       [ -2.48895215, -12.44778343,  12.49532827]]) # in MHz

# N5_D =  2*np.pi*np.array([[-2.94412424e-01, -5.68059200e-02, -1.02860888e+00],
#       [-5.68059200e-02, -5.40578469e-01, -2.67686240e-02],
#       [-1.02860888e+00, -2.67686240e-02,  5.05815320e+01]]) # in MHz
	  
# N1_D = 2*np.pi* np.array([[ 0.98491908,  3.28010265, -0.53784491],
#       [ 3.28010265, 25.88547678, -1.6335986 ],
#       [-0.53784491, -1.6335986 ,  1.41368001]]) # in MHz


# ErC_Dee =  np.array([[ 26.47042689, -55.90357828,  50.1679204 ],
#                             [-55.90357828, -20.86385225,  76.13493805],
#                              [ 50.1679204,  76.13493805,  -5.60657464]]) # in Mrad/s



# ErD_Dee = np.array([[ 11.08087889, -34.6687169,   12.14623706],
#                             [-34.6687169,  -33.09039672,  22.36229081],
#                             [ 12.14623706,  22.36229081,  22.00951783]]) #  in Mrad/s


In [ ]:
# b0 = 1.4 * 2*math.pi # Zeeman field strength in radians per microsecond

# krA = 1 # Rate constant 1/us
# krB = 0.1
# kf = 1
# kAB = 1e4
# kBA = 1e3

# tmax = 12. / krA # Maximum time us 
# tlist = np.linspace(0, tmax, math.ceil(1000*tmax)) # Time points for simulation

# theta = np.linspace(0, np.pi, 15)

# xyz = []

# for theta_ in theta:
#     x = np.sin(theta_)
#     y = 0
#     z = np.cos(theta_)
#     xyz.append([x, y, z])       

# oris = np.asarray(xyz)

# # Initialize arrays for latitude and longitude
# num_points = len(oris)
# lat = np.zeros(num_points)
# lon = np.zeros(num_points)

# # Convert Cartesian coordinates to latitude and longitude
# for i in range(num_points):
#     x, y, z = oris[i]
#     lat[i] = np.arcsin(z) * (180/np.pi)
#     lon[i] = np.arctan2(y, x) * (180/np.pi)

# dims = [2, 2, 3, 3, 3] # Dimensions of the system components (2 qubits, 1 spin-1 nucleus)
# dim = np.prod(dims) # Total dimension of the composite system

# H_C_list = []
# H_D_list = []

# for orientation in oris:
#     B0 = b0 * orientation  # Magnetic field vector along orientation

#     # Compute Hamiltonians for each orientation
#     Hzee = mkH1(dims, 0, B0) + mkH1(dims, 1, B0)  # Zeeman Hamiltonian for two spins
#     Hhfc_C = mkH12(dims, 0, 2, N5_C) + mkH12(dims, 1, 3, N1_C)
#     Hhfc_D = mkH12(dims, 0, 2, N5_D) + mkH12(dims, 1, 4, N1_D)
#     Hdee_C = mkH12(dims, 0, 1, ErC_Dee)
#     Hdee_D = mkH12(dims, 0, 1, ErD_Dee)
#     H0_C = Hzee + Hhfc_C + Hdee_C  # Total Hamiltonian for component C
#     H0_D = Hzee + Hhfc_D + Hdee_D  # Total Hamiltonian for component D
    
#     # Append Hamiltonians to the list
#     H_C_list.append(H0_C.data)
#     H_D_list.append(H0_D.data)

# # Now H_C_list and H_D_list contain Hamiltonians for each orientation

# Ps = 1/4 * mkSpinOp(dims,[]) - mkH12(dims, 0, 1, np.identity(3)) # Singlet projection operator


# rho0_C = (Ps / Ps.tr()).full().flatten()# Initial density matrix, normalized projection operator for the singlet state.
# rho0_D = np.zeros_like(rho0_C)
# Ps = Ps.data

# # Combine the initial states into one vector
# initial_state = np.concatenate((rho0_C, rho0_D)).flatten()

In [73]:
# # Convert the Hamiltonian H0 to a scipy sparse matrix and then to a PyTorch sparse tensor
# H_scipy = H0.data.tocsc()  # Convert to a SciPy sparse CSC matrix

# # Extract the indices and values for the sparse tensor
# H_indices = torch.LongTensor(np.vstack([H_scipy.nonzero()[0], H_scipy.nonzero()[1]]))  # Non-zero element indices
# H_values = torch.FloatTensor(H_scipy.data)  # Non-zero values
# H_shape = H_scipy.shape  # Shape of the matrix

# # Convert to a PyTorch sparse tensor
# # H_torch_sparse = torch.sparse_coo_tensor(H_indices, H_values, H_shape, requires_grad=True).to(device)
# H_torch = torch.tensor(H_scipy.toarray(), dtype=torch.float32, requires_grad=True)

# Ps = Ps.data.toarray()
# print(H_torch)


tensor([[ 154.5664,    0.0000,    0.0000,    4.3982,    0.0000,    0.0000,
            4.3982,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000],
        [   0.0000,    0.0000,    0.0000,    0.0000,    4.3982,    0.0000,
          -11.5515,    4.3982,    0.0000,    0.0000,    0.0000,    0.0000],
        [   0.0000,    0.0000, -154.5664,    0.0000,    0.0000,    4.3982,
            0.0000,  -11.5515,    4.3982,    0.0000,    0.0000,    0.0000],
        [   4.3982,    0.0000,    0.0000,  154.5664,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,    4.3982,    0.0000,    0.0000],
        [   0.0000,    4.3982,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000,    0.0000,    0.0000,  -11.5515,    4.3982,    0.0000],
        [   0.0000,    0.0000,    4.3982,    0.0000,    0.0000, -154.5664,
            0.0000,    0.0000,    0.0000,    0.0000,  -11.5515,    4.3982],
        [   4.3982,  -11.5515,    0.0000,    0.0000,    0.0000,    0.0000,
         -154.5664,

In [85]:
# Set device and data type
dtype = torch.float
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch.set_default_device(device)

# Define the neural network architecture
dimA, dimB = 2, 2  # Dimensions of subsystem A and B (for example 2x2 matrices)
ns = [20, 20, 20, 20]

net = nn.Sequential(
    nn.Linear(1, ns[0]),
    nn.Sigmoid(),
    *functools.reduce(operator.iconcat, [(nn.Linear(*d), nn.Sigmoid()) for d in zip(ns[:-1], ns[1:])], []),
    nn.Linear(ns[-1], 2*(dimA*dimA + dimB*dimB))  # Output real and imaginary parts for both density matrices
)

# Function to split the real and imaginary parts of the neural network output
def split_real_imag(u, dimA, dimB):
    lenA = dimA * dimA
    lenB = dimB * dimB
    rhoA_real = u[:, :lenA]
    rhoA_imag = u[:, lenA:2*lenA]
    rhoB_real = u[:, 2*lenA:2*lenA+lenB]
    rhoB_imag = u[:, 2*lenA+lenB:]
    return rhoA_real, rhoA_imag, rhoB_real, rhoB_imag

# Define the master equation function
def mesolve(t, u, P_s, HA, HB, krA, krB, kAB, kBA, kf, dimA, dimB):
    # Extract real and imaginary parts from the neural network output
    rhoA_real, rhoA_imag, rhoB_real, rhoB_imag = split_real_imag(u, dimA, dimB)
    
    # Reshape real and imaginary parts into matrix form
    rhoA_real = rhoA_real.reshape(-1, dimA, dimA)
    rhoA_imag = rhoA_imag.reshape(-1, dimA, dimA)
    rhoB_real = rhoB_real.reshape(-1, dimB, dimB)
    rhoB_imag = rhoB_imag.reshape(-1, dimB, dimB)

    # Separate real and imaginary components of Hamiltonian evolution: 
    # HA @ rhoA - rhoA @ HA and HB @ rhoB - rhoB @ HB
    HA_rhoA_real = HA @ rhoA_real - rhoA_real @ HA - rhoA_imag @ HA + HA @ rhoA_imag
    HA_rhoA_imag = HA @ rhoA_imag - rhoA_imag @ HA + rhoA_real @ HA - HA @ rhoA_real
    HB_rhoB_real = HB @ rhoB_real - rhoB_real @ HB - rhoB_imag @ HB + HB @ rhoB_imag
    HB_rhoB_imag = HB @ rhoB_imag - rhoB_imag @ HB + rhoB_real @ HB - HB @ rhoB_real
    
    # For the Lindblad terms (-kr * (P_s @ rho + rho @ P_s)/2)
    Ps_rhoA_real = P_s @ rhoA_real + rhoA_real @ P_s
    Ps_rhoA_imag = P_s @ rhoA_imag + rhoA_imag @ P_s
    Ps_rhoB_real = P_s @ rhoB_real + rhoB_real @ P_s
    Ps_rhoB_imag = P_s @ rhoB_imag + rhoB_imag @ P_s

    # drhoA/dt for real and imaginary parts
    drhoA_real = -HA_rhoA_imag - krA * Ps_rhoA_real / 2 - (kAB + kf) * rhoA_real + kBA * rhoB_real
    drhoA_imag = HA_rhoA_real - krA * Ps_rhoA_imag / 2 - (kAB + kf) * rhoA_imag + kBA * rhoB_imag
    
    # drhoB/dt for real and imaginary parts
    drhoB_real = -HB_rhoB_imag - krB * Ps_rhoB_real / 2 - (kBA + kf) * rhoB_real + kAB * rhoA_real
    drhoB_imag = HB_rhoB_real - krB * Ps_rhoB_imag / 2 - (kBA + kf) * rhoB_imag + kAB * rhoA_imag

    # Return concatenated real and imaginary parts as a flat vector
    return torch.cat((drhoA_real.flatten(), drhoA_imag.flatten(), drhoB_real.flatten(), drhoB_imag.flatten()), dim=-1)


In [86]:
# Generate the dudt function
def generate_dudt(HA, HB, P_s, krA, krB, kAB, kBA, kf, dimA, dimB):
    def dudt(u):
        return mesolve(None, u, P_s, HA, HB, krA, krB, kAB, kBA, kf, dimA, dimB)
    return dudt

# Generate a model for time evolution
def generate_model(net, u0):
    def fun(t):
        return torch.add(torch.mul(torch.add(torch.ones(1), torch.neg(torch.exp(torch.neg(t)))), net(t)), u0)
    return fun

# Define the loss function (to minimize)
def total_loss(u, u_t, u_t_exact, scale_norm):
    mse_dudt = nn.MSELoss()(u_t, u_t_exact)
    mse_norm = torch.mean((torch.norm(u, dim=1)-1)**2)
    return mse_dudt + scale_norm * mse_norm

# Generate loss function using autograd
def generate_loss(model, dudt, p=0):
    def fun(t):
        u = model(t)
        u_t = torch.autograd.functional.jacobian(lambda t_: model(t_).sum(axis=0), t, create_graph=True).squeeze(-1).T
        u_t_exact = dudt(u)
        return total_loss(u, u_t, u_t_exact, p)
    return fun


In [87]:
# Example Hamiltonians and parameters (change these values based on your system)
HA = torch.tensor([[0, 1], [1, 0]], dtype=dtype, device=device)  # Example 2x2 Hamiltonian for A
HB = torch.tensor([[0, 1], [1, 0]], dtype=dtype, device=device)  # Example 2x2 Hamiltonian for B
P_s = torch.tensor([[1, 0], [0, 0]], dtype=dtype, device=device)  # Some projection operator
krA, krB, kAB, kBA, kf = 0.1, 0.1, 0.2, 0.2, 0.05  # Example rates

# Generate the dudt function
dudt = generate_dudt(HA, HB, P_s, krA, krB, kAB, kBA, kf, dimA, dimB)

# Initial state (u0) in real and imaginary parts
u0_real = torch.randn(1, dimA*dimA + dimB*dimB, device=device)  # Initial real part
u0_imag = torch.randn(1, dimA*dimA + dimB*dimB, device=device)  # Initial imaginary part
u0 = torch.cat((u0_real, u0_imag), dim=1)

# Generate the model function for time evolution
model = generate_model(net, u0)

# Time points
t = torch.linspace(0, 10, 100, dtype=dtype, device=device).unsqueeze(-1)

# Generate the loss function
loss_fun = generate_loss(model, dudt)

# Optimizer for training
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

In [88]:
# Training loop
epochs = 10000
for epoch in range(epochs):
    optimizer.zero_grad()
    loss_value = loss_fun(t)
    loss_value.backward()
    optimizer.step()
    
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss_value.item()}')

/Users/sajai/opt/anaconda3/lib/python3.9/site-packages/torch/utils/_device.py:78: UserWarning: Using a target size (torch.Size([1600])) that is different to the input size (torch.Size([100, 16])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return func(*args, **kwargs)


RuntimeError: The size of tensor a (16) must match the size of tensor b (1600) at non-singleton dimension 1